<a href="https://colab.research.google.com/github/shlok-py/Pytorch_Tutorial/blob/main/Lesson_5_Building_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build the Neural Network
Neural networks comprise of layers/modules that perform operations on data. The torch.nn namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the nn.Module. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we’ll build a neural network to classify images in the FashionMNIST dataset.

In [2]:
import os
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training

In [3]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"On {device} device" )

On cpu device


## Define Class

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.ReLU_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.ReLU_stack(x)
        return logits

We create an instance of NeuralNetwork, and move it to the device, and print its structure.

In [9]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (ReLU_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the **model**, we pass it the input data. This executes the model’s forward, along with some background operations. Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with `dim=0` corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. We get the prediction probabilities by passing it through an instance of the `nn.Softmax` module.

In [22]:
X = torch.rand(1,28,28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(y_pred)

tensor([7])


## Breaking Down the neural Network Layers

In [23]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten

In [24]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear(in_features, out_features, bias)

In [25]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU()

In [26]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-1.6889e-01, -3.1071e-01,  4.6591e-01,  4.0720e-01, -4.9277e-02,
         -2.1237e-01,  1.9482e-01,  5.3131e-01, -2.2229e-01, -1.1351e-01,
          6.5878e-04, -2.7215e-01,  1.6171e-01,  1.7944e-01,  2.9506e-01,
         -4.1467e-01,  3.0076e-01,  7.7092e-01, -2.5137e-01, -5.7761e-01],
        [-7.5350e-02, -2.9079e-01,  3.7832e-01,  1.6469e-01, -1.1586e-01,
         -3.7963e-01, -5.6812e-03,  2.4956e-01, -2.4359e-01,  1.6988e-02,
          5.0036e-02, -1.4275e-01, -1.1193e-01,  7.0507e-01,  5.1315e-01,
         -4.1003e-01,  6.8458e-01,  6.5820e-01, -6.8402e-01, -6.5829e-01],
        [-1.5275e-01, -1.2506e-01,  3.1928e-01, -1.2465e-02, -3.1777e-01,
         -3.8849e-02,  4.7550e-02,  2.3371e-01, -1.2023e-01,  6.9762e-02,
         -1.5952e-01, -4.5169e-01,  2.4739e-01,  1.8292e-01,  2.5142e-01,
         -6.5961e-02, -1.3982e-01,  8.2140e-01, -2.3661e-01, -9.2508e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000e+00, 0.0000e+00, 4.6591e-01, 4.0

## nn.Sequential()

In [27]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [28]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)